In [1]:
!rm -rf ./logs/ 
%run Model.ipynb
%run dataset_loader.ipynb
import tensorflow as tf
#from dataset_loader import load_dataset


INPUT_WIDTH = 64
INPUT_HEIGHT = 64
INPUT_CHANNELS = 3

NUM_CLASSES = 5

LEARNING_RATE = 0.001   # Original value: 0.01
MOMENTUM = 0.9
KEEP_PROB = 1

EPOCHS = 1000
BATCH_SIZE = 64

train_sub_dir = '/data/image_format_small_dataset/*/*'
val_sub_dir = '/data/image_format_small_dataset_val/*/*'

test_sub_dir = '/data/image_format_small_dataset_test/*/*'

home_dir = os.path.dirname(os.path.realpath('__file__'))

train_dir = home_dir + train_sub_dir
val_dir = home_dir + val_sub_dir
test_dir = home_dir + test_sub_dir

training_dataset = load_dataset(data_dir= train_dir,minibatch_size=BATCH_SIZE)
val_dataset = load_dataset(data_dir= val_dir,minibatch_size=BATCH_SIZE)

test_dataset = load_dataset(data_dir = test_dir,minibatch_size=BATCH_SIZE)

experiment_no = 1
logdir = './log/alexnet/experiment'+str(experiment_no)



/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat

In [2]:

handle = tf.placeholder(tf.string, shape=[])


training_iterator = tf.data.Iterator.from_structure(training_dataset.output_types,training_dataset.output_shapes)
training_init_op = training_iterator.make_initializer(training_dataset)




Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.


In [3]:
iterator = tf.data.Iterator.from_string_handle(
    handle, training_iterator.output_types,training_dataset.output_shapes)

next_training_element = iterator.get_next()

In [4]:
with tf.device('/gpu:0'):
    alexnet = AlexNet(next_training_element,input_height=INPUT_HEIGHT,input_width=INPUT_WIDTH, num_classes=NUM_CLASSES)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    print('Training dataset...')
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    file_writer = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)
    
    training_string_handle =  sess.run(training_iterator.string_handle())
    
    summary_operation = tf.summary.merge_all()

    for epoch in range(EPOCHS):
        
        sess.run(training_init_op,feed_dict={handle:training_string_handle})
        _, accuracy,summary = sess.run([alexnet.training_operation,alexnet.accuracy_operation,summary_operation],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob,handle:training_string_handle})
       
      
        file_writer.add_summary(summary, epoch)
        print("accuracy for epoch {0}  :  {1}".format(epoch,accuracy))

    

Training dataset...
accuracy for epoch 0  :  0.265625
accuracy for epoch 1  :  0.28125
accuracy for epoch 2  :  0.1875
accuracy for epoch 3  :  0.1875
accuracy for epoch 4  :  0.15625
accuracy for epoch 5  :  0.125
accuracy for epoch 6  :  0.21875
accuracy for epoch 7  :  0.140625
accuracy for epoch 8  :  0.1875
accuracy for epoch 9  :  0.171875
accuracy for epoch 10  :  0.1875
accuracy for epoch 11  :  0.125
accuracy for epoch 12  :  0.140625
accuracy for epoch 13  :  0.25
accuracy for epoch 14  :  0.203125
accuracy for epoch 15  :  0.15625
accuracy for epoch 16  :  0.15625
accuracy for epoch 17  :  0.15625
accuracy for epoch 18  :  0.203125
accuracy for epoch 19  :  0.109375
accuracy for epoch 20  :  0.1875
accuracy for epoch 21  :  0.21875
accuracy for epoch 22  :  0.328125
accuracy for epoch 23  :  0.203125
accuracy for epoch 24  :  0.21875
accuracy for epoch 25  :  0.21875
accuracy for epoch 26  :  0.234375
accuracy for epoch 27  :  0.1875
accuracy for epoch 28  :  0.203125
accura

accuracy for epoch 237  :  0.109375
accuracy for epoch 238  :  0.15625
accuracy for epoch 239  :  0.171875
accuracy for epoch 240  :  0.1875
accuracy for epoch 241  :  0.21875
accuracy for epoch 242  :  0.1875
accuracy for epoch 243  :  0.171875
accuracy for epoch 244  :  0.203125
accuracy for epoch 245  :  0.171875
accuracy for epoch 246  :  0.15625
accuracy for epoch 247  :  0.140625
accuracy for epoch 248  :  0.296875
accuracy for epoch 249  :  0.1875
accuracy for epoch 250  :  0.21875
accuracy for epoch 251  :  0.265625
accuracy for epoch 252  :  0.234375
accuracy for epoch 253  :  0.078125
accuracy for epoch 254  :  0.203125
accuracy for epoch 255  :  0.171875
accuracy for epoch 256  :  0.140625
accuracy for epoch 257  :  0.1875
accuracy for epoch 258  :  0.265625
accuracy for epoch 259  :  0.15625
accuracy for epoch 260  :  0.265625
accuracy for epoch 261  :  0.140625
accuracy for epoch 262  :  0.1875
accuracy for epoch 263  :  0.140625
accuracy for epoch 264  :  0.15625
accuracy

In [ ]:

val_iterator = tf.data.Iterator.from_structure(val_dataset.output_types,val_dataset.output_shapes)
val_init_op = val_iterator.make_initializer(val_dataset)



In [ ]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    print('validation dataset...')
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(val_init_op)
    val_string_handle =  sess.run(val_iterator.string_handle())
    val_accuracy = sess.run([alexnet.accuracy_operation],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob, 
                                                                     handle:val_string_handle})
    

    print("accuracy for validation set :  {0}".format(val_accuracy))



In [ ]:

test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,test_dataset.output_shapes)
test_init_op = test_iterator.make_initializer(test_dataset)



In [ ]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    print('Testing dataset...')
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    sess.run(test_init_op)
    test_string_handle =  sess.run(test_iterator.string_handle())
    test_accuracy = sess.run([alexnet.accuracy_operation],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob, 
                                                                     handle:test_string_handle})
    

    print("accuracy for test set :  {0}".format(test_accuracy))

